In [ ]:
from google.colab import drive
import nbformat, os

drive.mount('/content/drive', force_remount=True)
SRC = '/content/drive/MyDrive/Colab Notebooks/Untitled17.ipynb'
DST = '/content/drive/MyDrive/colab_saves/predict difference.ipynb'

with open(SRC, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)


nb.metadata.pop('widgets', None)
for cell in nb.cells:
    cell.metadata.pop('widgets', None)
os.makedirs(os.path.dirname(DST), exist_ok=True)
with open(DST, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print("✅ Clean copy saved to", DST)

In [2]:
import pandas as pd
import pickle
from google.colab import drive


drive.mount('/content/drive')

load_path = "/content/drive/My Drive/colab_saves/all_data_clean.pkl"


with open(load_path, "rb") as f:
    all_data_clean = pickle.load(f)



Mounted at /content/drive


In [ ]:
print(all_data_clean.head())

    date_utc  n_titles                                             titles  \
0 2025-07-16         2  [MP Materials rips to all-time high after $500...   
1 2025-07-15         5  [MP Materials soars on $500M Apple deal backin...   
2 2025-07-14         1  [Apple's iPhone fold might sell for less than ...   
3 2025-07-11         1  [Top Seeking Alpha tech sector Quant picks ahe...   
4 2025-07-10         4  [Meta lured Apple’s Pang with pay package exce...   

                                           embedding  difference ticker  
0  [-0.0025580707, -0.012631489, -0.005138179, -0...    0.409533   AAPL  
1  [-0.004741954, -0.009954909, -0.002335629, -0....    1.188654   AAPL  
2  [-0.0031060807, -0.007945612, 0.0027573549, -0...    0.599332   AAPL  
3  [-0.0057898858, -0.008650952, -0.006357145, -0...   -1.228612   AAPL  
4  [-0.00744634, 0.004051717, -0.005791217, -0.00...   -1.837908   AAPL  


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


X = np.vstack(all_data_clean["embedding"].values)
y = all_data_clean["difference"].values


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state= 35)

max_dep = [2,5,10,20]
for i in max_dep:
    rf = RandomForestRegressor(
        n_estimators=200,
        max_depth=i,
        random_state=4,
        n_jobs=-1 )
    rf.fit(X_train, y_train)


    y_pred = rf.predict(X_test)


    mse  = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_test, y_pred)
    r2   = r2_score(y_test, y_pred)

    print("Random Forest Performance:")
    print(f"  MSE  : {mse:.4f}")
    print(f"  RMSE : {rmse:.4f}")
    print(f"  MAE  : {mae:.4f}")
    print(f"  R²   : {r2:.4f}")
    print(f"  max_depth   : {i:.4f}")



Random Forest Performance:
  MSE  : 20.9637
  RMSE : 4.5786
  MAE  : 2.3170
  R²   : -0.0491
  max_depth   : 2.0000
Random Forest Performance:
  MSE  : 21.4243
  RMSE : 4.6286
  MAE  : 2.3507
  R²   : -0.0721
  max_depth   : 5.0000
Random Forest Performance:
  MSE  : 21.9880
  RMSE : 4.6891
  MAE  : 2.3636
  R²   : -0.1003
  max_depth   : 10.0000
Random Forest Performance:
  MSE  : 21.8497
  RMSE : 4.6744
  MAE  : 2.3912
  R²   : -0.0934
  max_depth   : 20.0000


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


X = np.vstack(all_data_clean["embedding"].values)
y = all_data_clean["difference"].values


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state= 35)

rf = RandomForestRegressor(
        n_estimators=200,
        max_depth=2,
        random_state=4,
        n_jobs=-1 )
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)



import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, classification_report

#  Convert predictions to sign:  0, 1
def sign_to_class(x):
    if x > 0:
        return 1
    elif x <= 0:
        return 0


y_pred_class = np.array([sign_to_class(v) for v in y_pred])
y_true_class = np.array([sign_to_class(v) for v in y_test])


acc          = accuracy_score(y_true_class, y_pred_class)
f1_micro     = f1_score(y_true_class, y_pred_class, average="micro",    zero_division=0)
f1_macro     = f1_score(y_true_class, y_pred_class, average="macro",    zero_division=0)
f1_weighted  = f1_score(y_true_class, y_pred_class, average="weighted", zero_division=0)
rec_macro    = recall_score(y_true_class, y_pred_class, average="macro", zero_division=0)


print(f"  Accuracy     : {acc:.4%}")
print(f"  Micro-F1     : {f1_micro:.4%}")
print(f"  Macro-F1     : {f1_macro:.4%}")
print(f"  Weighted-F1  : {f1_weighted:.4%}")
print(f"  Macro-Recall : {rec_macro:.4%}")




  Accuracy     : 41.9048%
  Micro-F1     : 41.9048%
  Macro-F1     : 33.1489%
  Weighted-F1  : 29.2319%
  Macro-Recall : 49.0499%


In [ ]:
from sklearn.linear_model import Lasso

alphas = [0.01, 0.1, 0.5, 1, 5, 10]
results = []

for alpha in alphas:
    lasso = Lasso(alpha=alpha, max_iter=5000, random_state=34)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)

    mse  = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_test, y_pred)
    r2   = r2_score(y_test, y_pred)

    results.append((alpha, mse, rmse, mae, r2))

print("Lasso Regression Performance for different alpha values:")
print(" alpha |   MSE   |  RMSE  |   MAE   |   R²")
print("-----------------------------------------------")
for alpha, mse, rmse, mae, r2 in results:
    print(f"{alpha:6} | {mse:.4f} | {rmse:.4f} | {mae:.4f} | {r2:.4f}")

Lasso Regression Performance for different alpha values:
 alpha |   MSE   |  RMSE  |   MAE   |   R²
-----------------------------------------------
  0.01 | 12.2032 | 3.4933 | 2.0779 | -0.0055
   0.1 | 12.2329 | 3.4976 | 2.0729 | -0.0079
   0.5 | 12.2329 | 3.4976 | 2.0729 | -0.0079
     1 | 12.2329 | 3.4976 | 2.0729 | -0.0079
     5 | 12.2329 | 3.4976 | 2.0729 | -0.0079
    10 | 12.2329 | 3.4976 | 2.0729 | -0.0079


In [6]:

X = np.vstack(all_data_clean["embedding"].values)
y = all_data_clean["difference"].values


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state= 34)

lasso = Lasso(alpha=0.01, max_iter=5000, random_state=34)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)


y_pred_class = np.array([sign_to_class(v) for v in y_pred])
y_true_class = np.array([sign_to_class(v) for v in y_test])


acc          = accuracy_score(y_true_class, y_pred_class)
f1_micro     = f1_score(y_true_class, y_pred_class, average="micro",    zero_division=0)
f1_macro     = f1_score(y_true_class, y_pred_class, average="macro",    zero_division=0)
f1_weighted  = f1_score(y_true_class, y_pred_class, average="weighted", zero_division=0)
rec_macro    = recall_score(y_true_class, y_pred_class, average="macro", zero_division=0)


print(f"  Accuracy     : {acc:.4%}")
print(f"  Micro-F1     : {f1_micro:.4%}")
print(f"  Macro-F1     : {f1_macro:.4%}")
print(f"  Weighted-F1  : {f1_weighted:.4%}")
print(f"  Macro-Recall : {rec_macro:.4%}")

  Accuracy     : 50.4762%
  Micro-F1     : 50.4762%
  Macro-F1     : 36.6883%
  Weighted-F1  : 35.8442%
  Macro-Recall : 51.8519%


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb


X = np.vstack(all_data_clean["embedding"].values)
y = all_data_clean["difference"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

learn_rates = [0.01, 0.03, 0.05, 0.10, 0.30]

for lr in learn_rates:
    model = xgb.XGBRegressor(
        objective="reg:squarederror",
        learning_rate=lr,
        random_state=42,
        n_jobs=-1 )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse  = np.sqrt(mse)
    mae  = mean_absolute_error(y_test, y_pred)
    r2   = r2_score(y_test, y_pred)

    print(f"LR={lr:>5} | RMSE={rmse:.5f} | MAE={mae:.5f} | R²={r2:.5f} | MSE={mse:.5f}")




LR= 0.01 | RMSE=3.66979 | MAE=2.31566 | R²=-0.10965 | MSE=13.46734
LR= 0.03 | RMSE=3.64602 | MAE=2.39726 | R²=-0.09532 | MSE=13.29346
LR= 0.05 | RMSE=3.55720 | MAE=2.41120 | R²=-0.04261 | MSE=12.65368
LR=  0.1 | RMSE=3.61749 | MAE=2.43430 | R²=-0.07825 | MSE=13.08622
LR=  0.3 | RMSE=3.50080 | MAE=2.43955 | R²=-0.00981 | MSE=12.25560


In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

L1_grid = [0, 0.01, 0.1, 0.5]
L2_grid = [0.1, 1, 3, 10]

results = []

for a in L1_grid:
    for lam in L2_grid:
        model = xgb.XGBRegressor(
            objective="reg:squarederror",
            learning_rate= 0.3,
            reg_alpha=a,
            reg_lambda=lam,
            random_state=42,
            n_jobs=-1)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        rmse  = np.sqrt(mse)
        mae  = mean_absolute_error(y_test, y_pred)
        r2   = r2_score(y_test, y_pred)

        results.append({
            "reg_alpha": a,
            "reg_lambda": lam,
            "RMSE": rmse,
            "MAE": mae,
            "MSE": mse,
            "R2": r2
        })

        print(f"alpha={a:>4}, lambda={lam:>4} | RMSE={rmse:.5f} | MAE={mae:.5f} | R²={r2:.5f} | MSE={mse:.5f}")


results_df = pd.DataFrame(results).sort_values("RMSE").reset_index(drop=True)
print("\nTop combos by RMSE:")
print(results_df.head(10))



alpha=   0, lambda= 0.1 | RMSE=3.82090 | MAE=2.68030 | R²=-0.20292 | MSE=14.59929
alpha=   0, lambda=   1 | RMSE=3.50080 | MAE=2.43955 | R²=-0.00981 | MSE=12.25560
alpha=   0, lambda=   3 | RMSE=3.36569 | MAE=2.30491 | R²=0.06663 | MSE=11.32790
alpha=   0, lambda=  10 | RMSE=3.45737 | MAE=2.23818 | R²=0.01509 | MSE=11.95338
alpha=0.01, lambda= 0.1 | RMSE=3.73693 | MAE=2.65687 | R²=-0.15063 | MSE=13.96467
alpha=0.01, lambda=   1 | RMSE=3.44075 | MAE=2.37942 | R²=0.02454 | MSE=11.83875
alpha=0.01, lambda=   3 | RMSE=3.35453 | MAE=2.28362 | R²=0.07281 | MSE=11.25289
alpha=0.01, lambda=  10 | RMSE=3.46860 | MAE=2.25405 | R²=0.00868 | MSE=12.03121
alpha= 0.1, lambda= 0.1 | RMSE=3.80485 | MAE=2.61241 | R²=-0.19284 | MSE=14.47690
alpha= 0.1, lambda=   1 | RMSE=3.34278 | MAE=2.26523 | R²=0.07930 | MSE=11.17416
alpha= 0.1, lambda=   3 | RMSE=3.41614 | MAE=2.26955 | R²=0.03844 | MSE=11.67004
alpha= 0.1, lambda=  10 | RMSE=3.37053 | MAE=2.14992 | R²=0.06394 | MSE=11.36048
alpha= 0.5, lambda= 0.1 

In [19]:
model = xgb.XGBRegressor(
            objective="reg:squarederror",
            learning_rate= 0.3,
            reg_alpha=0.1,
            reg_lambda=1,
            random_state=42,
            n_jobs=-1)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

y_pred_class = np.array([sign_to_class(v) for v in y_pred])
y_true_class = np.array([sign_to_class(v) for v in y_test])


acc          = accuracy_score(y_true_class, y_pred_class)
f1_micro     = f1_score(y_true_class, y_pred_class, average="micro",    zero_division=0)
f1_macro     = f1_score(y_true_class, y_pred_class, average="macro",    zero_division=0)
f1_weighted  = f1_score(y_true_class, y_pred_class, average="weighted", zero_division=0)
rec_macro    = recall_score(y_true_class, y_pred_class, average="macro", zero_division=0)


print(f"  Accuracy     : {acc:.4%}")
print(f"  Micro-F1     : {f1_micro:.4%}")
print(f"  Macro-F1     : {f1_macro:.4%}")
print(f"  Weighted-F1  : {f1_weighted:.4%}")
print(f"  Macro-Recall : {rec_macro:.4%}")

  Accuracy     : 55.2381%
  Micro-F1     : 55.2381%
  Macro-F1     : 53.5529%
  Weighted-F1  : 54.1427%
  Macro-Recall : 54.2092%
